# Real Estate Data Analysis using SQL
This project aims to analyze real estate data in the USA using SQL and create an interactive Tableau dashboard to visualize the insights. The project provides an analysis of the real estate market, enabling users to explore various aspects such as property prices, trends, and geographical distribution.

# About Dataset
This dataset contains Real Estate listings in the US broken by State and zip code. 
The dataset has 1 CSV file with 10 columns:
* realtor-data.csv (200k+ entries)
* status (Housing status - a. ready for sale or b. ready to build)
* bed (# of beds)
* bath (# of bathrooms)
* acre_lot (Property / Land size in acres)
* city (city name)
* state (state name)
* zip_code (postal code of the area)
* house_size (house area/size/living space in square feet)
* prev_sold_date (Previously sold date)
* price (Housing price, it is either the current listing price or recently sold price if the house is sold recently)

# Visualization using Tableau
Full dashboard avaliable here https://public.tableau.com/app/profile/anastasiya.igonina/viz/USARealEstateDataAnalysis/Dashboard

In [4]:
%%html
<div class='tableauPlaceholder' id='viz1685372829356' style='position: relative'><noscript><a href='#'><img alt='USA Real Estate Data Analysis ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;US&#47;USARealEstateDataAnalysis&#47;Dashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='USARealEstateDataAnalysis&#47;Dashboard' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;US&#47;USARealEstateDataAnalysis&#47;Dashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1685372829356');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='1777px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# SQL Queries

In [5]:
# Load packages
import pandas as pd 
import sqlite3
from sqlalchemy import create_engine, text

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [6]:
# Load data
df = pd.read_csv('realtor-data.csv')

In [7]:
# Create a sqlite engine
engine = create_engine('sqlite://', echo=False)

# Export the dataframe as a table to the sqlite engine
df.to_sql("realtor", con =engine, index=False)

306000

In [8]:
# How many properties are listed for sale by state?
with engine.begin() as conn:
    query = text("""
    SELECT state,
    COUNT(*) AS properties_count 
    FROM realtor
    WHERE status = 'for_sale'
    GROUP BY state 
    ORDER BY properties_count DESC;
    """)
    df1 = pd.read_sql_query(query, conn)
    
df1

,state,properties_count
0,Massachusetts,169608
1,New Hampshire,46740
2,Rhode Island,28518
3,Puerto Rico,24679
4,Connecticut,14610
5,Maine,12687
6,Vermont,3801
7,Virgin Islands,2573
8,New York,1955
9,South Carolina,24


In [9]:
# Average number of bedrooms and bathrooms for properties that are ready to sale 
with engine.begin() as conn:
    query = text("""
    SELECT AVG(bed) AS avg_bedrooms,
    AVG(bath) AS avg_bathrooms
    FROM realtor
    WHERE status = 'for_sale';
    """)
    df2 = pd.read_sql_query(query, conn)
    
df2


,avg_bedrooms,avg_bathrooms
0,3.493992,2.574726


In [10]:
# Average, min and max prices by state
with engine.begin() as conn:
    query = text("""
    SELECT state, 
    ROUND(AVG(price)) AS avg_price, 
    MIN(price) AS min_price,
    MAX(price) AS max_price
    FROM realtor
    GROUP BY state
    ORDER BY state;
    """)
    df3 = pd.read_sql_query(query, conn)
    
df3

,state,avg_price,min_price,max_price
0,Connecticut,405391.0,14900.0,60000000.0
1,Maine,550056.0,14999.0,7900000.0
2,Massachusetts,992811.0,1.0,30000000.0
3,New Hampshire,494127.0,385.0,19500000.0
4,New Jersey,333490.0,333490.0,333490.0
5,New York,769929.0,14000.0,39500000.0
6,Puerto Rico,437137.0,15900.0,25000000.0
7,Rhode Island,500149.0,5500.0,14000000.0
8,South Carolina,18950.0,18950.0,18950.0
9,Tennessee,34900.0,34900.0,34900.0


In [11]:
# Median price by state
with engine.begin() as conn:
    query = text("""
    SELECT state, AVG(price) AS median_price
    FROM (
        SELECT state, price,
           ROW_NUMBER() OVER (PARTITION BY state ORDER BY price) AS row_num,
           COUNT(*) OVER (PARTITION BY state) AS total_count
        FROM realtor
    ) AS sub
    WHERE row_num IN ((total_count + 1) / 2, (total_count + 2) / 2)
    GROUP BY state;
    """)
    df4 = pd.read_sql_query(query, conn)
    
df4

,state,median_price
0,Connecticut,279000.0
1,Maine,375000.0
2,Massachusetts,615000.0
3,New Hampshire,360000.0
4,New Jersey,333490.0
5,New York,325000.0
6,Puerto Rico,145000.0
7,Rhode Island,379000.0
8,South Carolina,18950.0
9,Tennessee,34900.0


In [12]:
# Average price for square feet for different states
with engine.begin() as conn:
    query = text("""
    SELECT state,
    ROUND(AVG(price / house_size)) AS avg_price_per_sqft 
    FROM realtor
    WHERE house_size IS NOT NULL
    GROUP BY state 
    ORDER BY state;
    """)
    df5 = pd.read_sql_query(query, conn)
    
df5

,state,avg_price_per_sqft
0,Connecticut,197.0
1,Maine,335.0
2,Massachusetts,509.0
3,New Hampshire,267.0
4,New Jersey,222.0
5,New York,334.0
6,Puerto Rico,187.0
7,Rhode Island,264.0
8,Vermont,178.0
9,Virgin Islands,354.0


In [13]:
# Top 10 most expensive cities by average price
with engine.begin() as conn:
    query = text("""
    SELECT city, ROUND(AVG(price),0) AS avg_price
    FROM realtor
    GROUP BY city
    ORDER BY avg_price DESC
    LIMIT 10;
    """)
    df6 = pd.read_sql_query(query, conn)
    
df6

,city,avg_price
0,Waterfront,12000000.0
1,Woods Hole,8583000.0
2,Chilmark,7136333.0
3,Siasconset,6495000.0
4,Tisbury,6406333.0
5,Nantucket,5550593.0
6,Hyannis Port,5382167.0
7,Aquinnah,5119286.0
8,Montauk,4704595.0
9,Weston,4641733.0


In [14]:
# List of properties that is more expensive than average price in this city
with engine.begin() as conn:
    query = text("""
    WITH city_avg_price AS (
        SELECT city, ROUND(AVG(price)) AS avg_price
        FROM realtor
        GROUP BY city
    )
    SELECT rd.*,
           cap.avg_price AS city_average_price
    FROM realtor rd
    JOIN city_avg_price cap ON rd.city = cap.city
    WHERE rd.price > cap.avg_price;
    """)
    df7 = pd.read_sql_query(query, conn)
    
df7

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,city_average_price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,None,105000.0,92879.0
1,for_sale,5.0,3.0,7.46,Las Marias,Puerto Rico,670.0,5403.0,None,300000.0,173559.0
2,for_sale,3.0,2.0,0.08,Juana Diaz,Puerto Rico,795.0,1045.0,None,150000.0,115241.0
3,for_sale,5.0,5.0,0.74,Ponce,Puerto Rico,731.0,2677.0,None,649000.0,175336.0
4,for_sale,4.0,4.0,0.22,Mayaguez,Puerto Rico,680.0,3450.0,None,235000.0,176220.0
...,...,...,...,...,...,...,...,...,...,...,...
107191,for_sale,4.0,4.0,140.33,Turner,Maine,4282.0,5268.0,None,1850000.0,356850.0
107192,for_sale,6.0,4.0,0.62,Auburn,Maine,4210.0,6000.0,2004-02-18,750000.0,425403.0
107193,for_sale,3.0,3.0,1.21,Farmingdale,Maine,4344.0,2200.0,None,459000.0,306000.0
107194,for_sale,2.0,1.0,495.00,Mount Vernon,Maine,4352.0,1200.0,2018-01-29,949999.0,363431.0


In [15]:
# Find mean, min and max houses sizes by state
with engine.begin() as conn:
    query = text("""
    SELECT state, 
    ROUND(AVG(house_size),0) AS avg_size, 
    MIN(house_size) AS min_size,
    MAX(house_size) AS max_size
    FROM realtor
    WHERE house_size IS NOT NULL
    GROUP BY state
    ORDER BY state;
    """)
    df8 = pd.read_sql_query(query, conn)
    
df8

,state,avg_size,min_size,max_size
0,Connecticut,2007.0,100.0,99999.0
1,Maine,2131.0,225.0,35388.0
2,Massachusetts,2284.0,104.0,99999.0
3,New Hampshire,2249.0,273.0,21500.0
4,New Jersey,1500.0,1500.0,1500.0
5,New York,2837.0,418.0,34000.0
6,Puerto Rico,1942.0,181.0,1450112.0
7,Rhode Island,2168.0,207.0,22774.0
8,Vermont,2393.0,160.0,12845.0
9,Virgin Islands,3941.0,235.0,49658.0


In [16]:
# Properties with latest sold date for each city
with engine.begin() as conn:
    query = text("""
    WITH earliest_sold_properties AS (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY city ORDER BY prev_sold_date DESC) AS property_rank
        FROM realtor
        WHERE prev_sold_date IS NOT NULL
    )
    SELECT *
    FROM earliest_sold_properties
    WHERE property_rank = 1
    ORDER BY city, prev_sold_date DESC;
    """)
    df9 = pd.read_sql_query(query, conn)
    
df9

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,property_rank
0,for_sale,2.0,1.0,NaN,Abington,Massachusetts,2351.0,896.0,2007-05-31,269000.0,1
1,for_sale,2.0,2.0,NaN,Acton,Massachusetts,1718.0,1338.0,2019-02-19,380000.0,1
2,for_sale,NaN,NaN,1.72,Acushnet,Massachusetts,2743.0,NaN,2005-11-08,239900.0,1
3,for_sale,2.0,2.0,0.28,Adams,Massachusetts,1220.0,1206.0,2011-10-28,159900.0,1
4,for_sale,2.0,2.0,NaN,Agawam,Massachusetts,1001.0,1056.0,2019-09-12,175000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
720,for_sale,3.0,1.0,1.49,Wrentham,Massachusetts,2093.0,1044.0,2010-01-22,430000.0,1
721,for_sale,NaN,NaN,78.66,Yabucoa,Puerto Rico,767.0,NaN,2021-03-15,380000.0,1
722,for_sale,2.0,2.0,NaN,Yarmouth,Massachusetts,2673.0,1239.0,2018-12-20,390000.0,1
723,for_sale,2.0,2.0,NaN,Yarmouth Port,Massachusetts,2675.0,1092.0,2020-03-06,179000.0,1


In [17]:
# Create categories for bedrooms and count amount of properties in each category
with engine.begin() as conn:
    query = text("""
    SELECT bedroom_category, COUNT(*) AS property_count
    FROM (
        SELECT *,
               CASE
                   WHEN bed BETWEEN 1 AND 2 THEN 'small'
                   WHEN bed BETWEEN 2 AND 3 THEN 'medium'
                   ELSE 'large'
               END AS bedroom_category
        FROM realtor
    ) subquery
    GROUP BY bedroom_category;
    """)
    df10 = pd.read_sql_query(query, conn)
    
df10

,bedroom_category,property_count
0,large,145885
1,medium,87316
2,small,72799


In [18]:
# Percentage of expensive properties for each number of bathrooms
with engine.begin() as conn:
    query = text("""
    SELECT bath,
       COUNT(*) AS property_count,
       AVG(CASE WHEN price > 500000 THEN 1.0 ELSE 0.0 END) * 100 AS percentage_expensive_properties
    FROM realtor
    WHERE bath IS NOT NULL
    GROUP BY bath
    ORDER BY bath;
    """)
    df11 = pd.read_sql_query(query, conn)
    
df11

,bath,property_count,percentage_expensive_properties
0,1.0,52578,26.069839
1,2.0,96159,39.496043
2,3.0,62183,68.518727
3,4.0,21770,81.162150
4,5.0,8289,86.789721
5,6.0,4491,88.532621
6,7.0,1962,93.883792
7,8.0,1455,97.731959
8,9.0,867,93.425606
9,10.0,531,95.103578


In [19]:
# What is the cumulative sum of the house sizes for properties in each combination of city and state, ordered by the cumulative sum in descending order?
with engine.begin() as conn:
    query = text("""
    SELECT DISTINCT city, state, 
           SUM(house_size) OVER (PARTITION BY city, state) AS cumulative_sum
    FROM realtor
    WHERE house_size IS NOT NULL
    ORDER BY cumulative_sum DESC;
    """)
    df12 = pd.read_sql_query(query, conn)
    
df12

,city,state,cumulative_sum
0,Boston,Massachusetts,56254172.0
1,Providence,Rhode Island,11043729.0
2,San Juan,Puerto Rico,8907743.0
3,Newton,Massachusetts,8217274.0
4,Worcester,Massachusetts,8150672.0
...,...,...,...
1133,Scotland,Connecticut,960.0
1134,Porter,Maine,912.0
1135,Wardsboro,Vermont,800.0
1136,Isla Verde,Puerto Rico,700.0


In [20]:
# Calculate the price difference between consecutive sales within each state, excluding duplicated rows
with engine.begin() as conn:
    query = text("""
    SELECT state, prev_sold_date, price,
           price - LAG(price) OVER (PARTITION BY state ORDER BY prev_sold_date) AS price_difference
    FROM (
        SELECT state, prev_sold_date, price,
               ROW_NUMBER() OVER (PARTITION BY state, price, prev_sold_date ORDER BY price) AS row_num
        FROM realtor
        WHERE prev_sold_date IS NOT NULL
        ) sub
    WHERE row_num = 1
    ORDER BY state, prev_sold_date;
    """)
    df13 = pd.read_sql_query(query, conn)
    
df13

,state,prev_sold_date,price,price_difference
0,Connecticut,1965-03-12,309900.0,NaN
1,Connecticut,1966-03-09,299900.0,-10000.0
2,Connecticut,1966-06-06,353900.0,54000.0
3,Connecticut,1968-04-01,184900.0,-169000.0
4,Connecticut,1973-03-14,650000.0,465100.0
...,...,...,...,...
9877,Virgin Islands,2010-01-26,129000.0,NaN
9878,Virgin Islands,2013-10-11,950000.0,821000.0
9879,Virgin Islands,2016-04-28,79500.0,-870500.0
9880,Virgin Islands,2016-11-16,176500.0,97000.0


### Thank you for reading!